In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 22.8 MB/s eta 0:00:00


Te amo


In [ ]:
from google.colab import files

uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))
pdf_path = list(uploaded.keys())[0]


Saving Electronica1.pdf to Electronica1.pdf
User uploaded file "Electronica1.pdf" with length 242553 bytes


In [ ]:
!pip install pdfminer.six


In [ ]:
!pip install pdfminer.six unidecode


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 4.0 MB/s eta 0:00:00


In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode
import re
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def extract_section_with_title_and_table(keyword):
    def extract_table_column(table, keyword):
        col_index = None
        for idx, row in enumerate(table):
            uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
            if uppercase_columns:
                matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                if matching_columns:
                    initial_col_index = matching_columns[0]
                    subsequent_rows = table[idx+1:]  # Rows after the header

                    # Check if the subsequent rows of the initial column are empty or "None"
                    if all(str(row[initial_col_index]).strip() in ["", "None"] for row in subsequent_rows):
                        # Look for the next column that has content
                        for alt_col_index in range(initial_col_index + 1, len(row)):
                            if any(row[alt_col_index].strip() not in ["", "None"] for row in subsequent_rows):
                                col_index = alt_col_index
                                break
                    else:
                        col_index = initial_col_index
                    break
        return col_index

    # Primero, buscaremos la palabra clave en el texto
    text = extract_text(pdf_path)
    lines = text.split('\n')

    section_content = []
    capture_content = False

    for line in lines:
        line_without_accent = unidecode(line)

        if keyword in line_without_accent:
            capture_content = True

        if capture_content:
            section_content.append(line)

        # Si encontramos una línea completamente en mayúsculas después de la keyword, paramos
        if capture_content and line_without_accent.isupper() and keyword not in line_without_accent:
            break

    if section_content:
        return section_content

    # Si no se encontró la palabra clave en el texto, buscar en las tablas
    content_from_table = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                col_index = extract_table_column(table, keyword)
                if col_index is not None:
                    for row in table:
                        if len(row) > col_index:
                            content_from_table.append(row[col_index])

    return content_from_table

# Palabra clave que deseas buscar en el PDF
keyword = "PROGRAMA SINTÉTICO"

# Extraer la sección con la palabra clave y mostrar las salidas
section = extract_section_with_title_and_table(keyword)
for line in section:
    print(line)


PROGRAMA SINTÉTICO:
Introducción a los semiconductores,
Diodo semiconductor
Transistores de Unión Bipolar (BJT’S)
Transistores de efecto de campo (FET’S)
Realimentación Colector-base
Transistor en switching
None


In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode
import re
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def starts_with_roman_numeral(s):
    """Determina si una cadena comienza con un numeral romano."""
    roman_numerals = ["I ", "I. ", "II ", "II. ", "III ", "III. ", "IV ", "IV. ",
                      "V ", "V. ", "VI ", "VI. ", "VII ", "VII. ", "VIII ", "VIII. ",
                      "IX ", "IX. ", "X ", "X. "]
    for numeral in roman_numerals:
        if s.startswith(numeral):
            return True
    return False

def extract_section_with_title_and_table(keyword):
    def extract_table_column(table, keyword):
        col_index = None
        for idx, row in enumerate(table):
            uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
            if uppercase_columns:
                matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                if matching_columns:
                    col_index = matching_columns[0]
                    break
        return col_index

    text = extract_text(pdf_path)
    lines = text.split('\n')
    section_content = []
    capture_content = False

    for line_num, line in enumerate(lines):
        line_without_accent = unidecode(line)

        if keyword in line_without_accent:
            capture_content = True

        if capture_content:
            section_content.append(line)

        # Si encontramos un título que comienza con un numeral romano
        if capture_content and starts_with_roman_numeral(line_without_accent):
            # Verificamos la siguiente línea
            next_line = lines[line_num + 1] if line_num + 1 < len(lines) else ""
            if next_line.isupper() or starts_with_roman_numeral(next_line):
                break

        # Si encontramos una línea completamente en mayúsculas después de la keyword, paramos
        if capture_content and line_without_accent.isupper() and keyword not in line_without_accent:
            break

    if section_content:
        return section_content

    content_from_table = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                col_index = extract_table_column(table, keyword)
                if col_index is not None:
                    for row in table:
                        if len(row) > col_index:
                            content_from_table.append(row[col_index])

    return content_from_table

# Palabra clave que deseas buscar en el PDF
keyword = "PROGRAMA SINTÉTICO"

# Extraer la sección con la palabra clave y mostrar las salidas
section = extract_section_with_title_and_table(keyword)
for line in section:
    print(line)



PROGRAMA SINTÉTICO:
Introducción a los semiconductores,
Diodo semiconductor
Transistores de Unión Bipolar (BJT’S)
Transistores de efecto de campo (FET’S)
Realimentación Colector-base
Transistor en switching
None


In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode
import re
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def extract_section_with_title_and_table(keyword):
    def extract_table_column(table, keyword):
        col_index = None
        for idx, row in enumerate(table):
            uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
            if uppercase_columns:
                matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                if matching_columns:
                    col_index = matching_columns[0]
                    break
        return col_index

    # Primero, buscaremos la palabra clave en el texto
    text = extract_text(pdf_path)
    lines = text.split('\n')

    section_content = []
    capture_content = False

    for line in lines:
        line_without_accent = unidecode(line)

        if keyword in line_without_accent:
            capture_content = True

        if capture_content:
            section_content.append(line)

        # Si encontramos una línea completamente en mayúsculas después de la keyword, paramos
        if capture_content and line_without_accent.isupper() and keyword not in line_without_accent:
            break

    if section_content:
        return section_content

    # Si no se encontró la palabra clave en el texto, buscar en las tablas
    content_from_table = []

    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                col_index = extract_table_column(table, keyword)
                if col_index is not None:
                    for row in table:
                        if len(row) > col_index:
                            content_from_table.append(row[col_index])

    return content_from_table

# Palabra clave que deseas buscar en el PDF
keyword = "PROGRAMA SINTÉTICO"

# Extraer la sección con la palabra clave y mostrar las salidas
section = extract_section_with_title_and_table(keyword)
for line in section:
    print(line)






PROGRAMA SINTÉTICO:
FUENTES DE CORRIENTE Y SUS APLICACIONES
EL BJT-FET EN ALTA FRECUENCIA
AMPLIFICADORES REALIMENTADOS
AMPLIFICADORES DE SEÑAL GRANDE.
III. ESTRATEGIAS
Metodología Pedagógica y Didáctica:
1. El docente indicara a los estudiantes leer el tema de la siguiente sesion, sualmente
incluido dentro de la bibliografía dada.
2. Desarrollo de los temas utilizando los recursos del aula, material impreso.
3. Elaboración de ejercicios de aplicación ( tareas, consultas en la WEB, etc) incluyendo
simulaciones utilizando software libre.
4. Se facilitara a los estudiantes el material escrito necesario para llevar a cabo las
practicas de laboratorio.
5. Se motivara la creatividad por medio de un proyecto practico donde los estudiantes
aplicaran los conceptos vistos.
6. Se motivara la consulta de los estudiantes al recomendar la lectura de los temas vistos
para una discusión posterior que les permita visualizar diversos puntos de vista de los
autores.
Horas Horas Horas Total Horas Créditos

In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode
import re

def extract_section_with_title(pdf_path):
    text = extract_text(pdf_path)
    lines = text.split('\n')

    # Lista de palabras clave
    keywords = ["PROGRAMA SINTETICO", "UNIDADES", "TEMAS"]

    # Patrón para identificar números romanos seguido de cualquier texto
    roman_pattern = re.compile(r'^(I{1,3}|IV|V?I{0,3}|IX|X{1,3})\.\s+')

    section_content = []
    capture_content = False

    for line in lines:
        line_without_accent = unidecode(line)

        # Ignora líneas vacías (saltos de página)
        if not line.strip():
            continue

        if capture_content:
            roman_match = roman_pattern.match(line_without_accent)
            if roman_match:
                # Extractamos la parte después del número romano
                after_roman = line_without_accent[roman_match.end():].split()
                if after_roman and after_roman[0].isupper():
                    break
            elif any(keyword in line_without_accent for keyword in keywords) or line_without_accent.isupper():
                break

        if not capture_content:
            for keyword in keywords:
                if keyword in line_without_accent:
                    capture_content = True
                    break

        if capture_content:
            section_content.append(line)

    return section_content

section = extract_section_with_title(pdf_path)
for line in section:
    print(line)


PROGRAMA SINTÉTICO: 


In [ ]:
!pip install pdfminer.six


In [ ]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 36.4 MB/s eta 0:00:00


In [ ]:
pip install pdfplumber

In [ ]:
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def extract_column_with_uppercase_keyword(pdf_path, keyword):
    with pdfplumber.open(pdf_path) as pdf:
        all_data = []
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                for idx, row in enumerate(table):
                    # Identificar si la fila actual tiene celdas en mayúsculas (potenciales encabezados)
                    uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
                    if uppercase_columns:
                        # Verificar si el keyword está en las celdas identificadas
                        matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                        if matching_columns:
                            col_index = matching_columns[0]
                            # Recopilar datos de la columna especificada de las filas subsiguientes
                            for subsequent_row in table[idx+1:]:
                                all_data.append(subsequent_row[col_index])
                            break  # Rompemos el ciclo una vez que encontramos el keyword
                #print("Encabezado detectado:", table[0])  # Imprimir encabezado para revisión
        return all_data

# Extraer información de la columna con encabezado en mayúsculas "PROGRAMA SINTÉTICO"
data = extract_column_with_uppercase_keyword(pdf_path, "PROGRAMA SINTÉTICO")

# Imprimir la información extraída
for item in data:
    print(item)


III. ESTRATEGIAS
Metodología Pedagógica y Didáctica:
1. El docente indicara a los estudiantes leer el tema de la siguiente sesion, sualmente
incluido dentro de la bibliografía dada.
2. Desarrollo de los temas utilizando los recursos del aula, material impreso.
3. Elaboración de ejercicios de aplicación ( tareas, consultas en la WEB, etc) incluyendo
simulaciones utilizando software libre.
4. Se facilitara a los estudiantes el material escrito necesario para llevar a cabo las
practicas de laboratorio.
5. Se motivara la creatividad por medio de un proyecto practico donde los estudiantes
aplicaran los conceptos vistos.
6. Se motivara la consulta de los estudiantes al recomendar la lectura de los temas vistos
para una discusión posterior que les permita visualizar diversos puntos de vista de los
autores.
Horas Horas Horas Total Horas Créditos
profesor/semana Estudiante/semana Estudiante/semestre
Tipo de Curso TD TC TA (TD + TC) (TD + TC +TA) X 16 semanas
4 2 2 6 8 128 3
Trabajo Presencial D

In [ ]:
import pdfplumber

def is_uppercase(string):
    return string.isupper()

def extract_column_with_uppercase_keyword(pdf_path, keyword):
    with pdfplumber.open(pdf_path) as pdf:
        all_data = []
        capture_next_page = False
        col_index = None
        for page in pdf.pages:
            tables = page.extract_tables()
            for idx, table in enumerate(tables):
                if capture_next_page and col_index is not None:
                    # Si se espera que la tabla continúe desde la página anterior
                    for row in table:
                        all_data.append(row[col_index])
                    capture_next_page = False if idx < len(tables) - 1 else True
                    continue

                for r_idx, row in enumerate(table):
                    uppercase_columns = [i for i, cell in enumerate(row) if is_uppercase(str(cell))]
                    if uppercase_columns:
                        matching_columns = [i for i in uppercase_columns if keyword in str(row[i])]
                        if matching_columns:
                            col_index = matching_columns[0]
                            for subsequent_row in table[r_idx+1:]:
                                all_data.append(subsequent_row[col_index])
                            capture_next_page = True if idx == len(tables) - 1 else False
                            break
            #print("Encabezado detectado en la página:", page.page_number)  # Imprimir número de página para revisión

        return all_data

# Extraer información de la columna con encabezado en mayúsculas "PROGRAMA SINTÉTICO"
data = extract_column_with_uppercase_keyword(pdf_path, "PROGRAMA SINTÉTICO")

# Imprimir la información extraída
for item in data:
    print(item)


III. ESTRATEGIAS
Metodología Pedagógica y Didáctica:
1. El docente indicara a los estudiantes leer el tema de la siguiente sesion, sualmente
incluido dentro de la bibliografía dada.
2. Desarrollo de los temas utilizando los recursos del aula, material impreso.
3. Elaboración de ejercicios de aplicación ( tareas, consultas en la WEB, etc) incluyendo
simulaciones utilizando software libre.
4. Se facilitara a los estudiantes el material escrito necesario para llevar a cabo las
practicas de laboratorio.
5. Se motivara la creatividad por medio de un proyecto practico donde los estudiantes
aplicaran los conceptos vistos.
6. Se motivara la consulta de los estudiantes al recomendar la lectura de los temas vistos
para una discusión posterior que les permita visualizar diversos puntos de vista de los
autores.
Horas Horas Horas Total Horas Créditos
profesor/semana Estudiante/semana Estudiante/semestre
Tipo de Curso TD TC TA (TD + TC) (TD + TC +TA) X 16 semanas
4 2 2 6 8 128 3
Trabajo Presencial D

In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode

def visualize_keyword_context(pdf_path):
    text = extract_text(pdf_path)
    lines = text.split('\n')

    keywords = ["PROGRAMA SINTETICO", "UNIDADES", "TEMAS"]

    for i, line in enumerate(lines):
        line_without_accent = unidecode(line)

        if any(keyword in line_without_accent for keyword in keywords):
            # Imprimir 5 líneas antes y 5 después de la línea con la palabra clave
            start = max(0, i - 5)
            end = min(len(lines), i + 6)

            for j in range(start, end):
                print(lines[j])
            print('-'*50)  # Separador para cada bloque

visualize_keyword_context(pdf_path)


 
 
 
 
 
PROGRAMA SINTÉTICO: 

FUENTES DE CORRIENTE Y SUS APLICACIONES 
EL BJT-FET EN ALTA FRECUENCIA 
AMPLIFICADORES REALIMENTADOS 
AMPLIFICADORES DE SEÑAL GRANDE. 
--------------------------------------------------
 
V. ORGANIZACIÓN / TIEMPOS 

Espacios, Tiempos, Agrupamientos: 

PROGRAMA    POR    UNIDADES:   Cada   semana   se   compone   de   cuatro   horas   teoricas 
presenciales  ,  dos  horas  practicas  en  laboratorio    y  ocho  horas  de  trabajo  autonomo.  Las 
sesiones  de  laboratorio  se  hacen  en  grupos  de  tres  estudiantes  por  banco  de  trabajo  para  un 
total de 18 estudiantes por sala. 

La  primera  unidad(fuentes  de  corriente  y  sus  aplicaciones)  se  cubrira  en  un  lapso  de  tres 
--------------------------------------------------


In [ ]:
from pdfminer.high_level import extract_text
from unidecode import unidecode

def extract_section_with_title(pdf_path):
    text = extract_text(pdf_path)
    lines = text.split('\n')

    # Lista de palabras clave
    keywords = ["PROGRAMA SINTETICO", "UNIDADES", "TEMAS"]

    section_content = []
    capture_content = False
    column_index = None

    for line in lines:
        line_without_accent = unidecode(line)

        # Ignora líneas vacías (saltos de página)
        if not line.strip():
            continue

        if not capture_content:
            parts = line_without_accent.split("  ")
            for keyword in keywords:
                if keyword in parts:
                    column_index = parts.index(keyword)
                    capture_content = True
                    keywords.remove(keyword)  # Eliminamos la palabra clave encontrada de la lista
                    break

        elif capture_content and column_index is not None:
            parts = line_without_accent.split("  ")
            if len(parts) > column_index:
                section_content.append(parts[column_index].strip())
            else:
                # Si la estructura de la tabla parece haber terminado, termina la captura
                break

    return section_content

section = extract_section_with_title(pdf_path)
for line in section:
    print(line)
